In [1]:
import numpy as np
import h5py

import sys
from joblib import Parallel,delayed
import time

import re
import os

In [2]:
raw_data_path = '../../Processed_Data/'

In [3]:
def sort_data_by_pattern(filename,relevant_patterns,cut_start=True,max_length=-1):
    
    t0 = time.time()
    
    # Open the input/output file
    filename_in = raw_data_path + filename
    f = h5py.File(filename_in,'r')
    f_out = h5py.File(filename,'w')
    
    # Go through all 15 circuits
    for circuit in range(15): 
        
        # Create the circuit group of current circuit
        circuit_group = f_out.create_group('Circuit_' + str(circuit)) 
        
        if cut_start:
            ids      = np.array(f['Circuit_14/Patterns'])[:,1]
            id_diff  = ids[1:] - ids[:-1]
            id_diff  = (128+id_diff)%128
            id_diff  = id_diff[2:] + id_diff[1:-1] + id_diff[0:-2]
            id_diff  = np.logical_and(id_diff>=2,id_diff<=4)
            id_start = np.argmax(id_diff != True)+3
        else:
            id_start = 0
            
        if max_length == -1:
            id_stop = np.array(f['Circuit_14/Patterns'])[-1,0]
        else:
            id_stop = max_length
        
        # For each of the relevant patterns, distill the data
        for pattern_id in relevant_patterns: 

            # This tells us, which stim ids are relevant
            mask = id_start + np.where(np.array(f['Circuit_' + str(circuit) + '/Patterns'])[id_start:id_stop,1] == pattern_id)[0] 

            # Get spike data that has been masked by mask
            data = np.array(f['Circuit_' + str(circuit) + '/Spikes'])
            vec  = data[:,0]
            try:
                elements = np.concatenate([np.argwhere(vec == i) for i in mask])
            except:
                break
            data = data[elements,:]

            # Save the data in the file
            pattern_group = circuit_group.create_group('Pattern_'+str(pattern_id))
            pattern_group.create_dataset("Stim_IDs",data=mask,dtype=np.uint32)
            pattern_group.create_dataset("Spikes",data=data,dtype=np.uint32) #Was a bug: uint16: 200819
            
    # Close files again
    f_out.close()
    f.close()
        
    print(filename,time.time()-t0,'sec')
    sys.stdout.flush()

## Standard experiment

In [4]:
filenames = []
for (_,_,files) in os.walk(raw_data_path):
    for filename in files:
        f_no_numbers = re.sub(r'[0-9]+','',filename)
        if f_no_numbers == 'MEA__DIV_.h':
            filenames.append(filename)
        elif f_no_numbers == 'MEA__DIV__run_.h':
            filenames.append(filename)

In [5]:
relevant_patterns = list(np.concatenate([np.array([0,1,3]),np.arange(10,70)]))

In [6]:
Parallel(n_jobs=10)(delayed(sort_data_by_pattern)(filename,relevant_patterns,True,75000) for filename in filenames)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Spike Shape

In [7]:
filenames = []
for (_,_,files) in os.walk(raw_data_path):
    for filename in files:
        f_no_numbers = re.sub(r'[0-9]+','',filename)
        if f_no_numbers == 'MEA__DIV_spike_shape.h':
            filenames.append(filename)
        elif f_no_numbers == 'MEA__DIV_spike_shape_.h':
            filenames.append(filename)

In [8]:
relevant_patterns = [0,1]

In [9]:
Parallel(n_jobs=10)(delayed(sort_data_by_pattern)(filename,relevant_patterns,True,175000) for filename in filenames)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Long Stim

In [10]:
filenames = []
for (_,_,files) in os.walk(raw_data_path):
    for filename in files:
        f_no_numbers = re.sub(r'[0-9]+','',filename)
        if f_no_numbers == 'MEA__DIV_long_stim.h':
            filenames.append(filename)

In [11]:
relevant_patterns = [0,1,3]

In [12]:
Parallel(n_jobs=10)(delayed(sort_data_by_pattern)(filename,relevant_patterns,True,-1) for filename in filenames)

[None, None, None, None]

## Circular Timing

In [4]:
filenames = []
for (_,_,files) in os.walk(raw_data_path):
    for filename in files:
        f_no_numbers = re.sub(r'[0-9]+','',filename)
        if f_no_numbers == 'MEA__DIV_circular_timing.h':
            filenames.append(filename)

In [5]:
relevant_patterns = [0,80]

In [6]:
Parallel(n_jobs=10)(delayed(sort_data_by_pattern)(filename,relevant_patterns,True,-1) for filename in filenames)

[None, None, None, None, None, None, None, None, None, None, None, None]

## Stim Addition

In [7]:
filenames = []
for (_,_,files) in os.walk(raw_data_path):
    for filename in files:
        f_no_numbers = re.sub(r'[0-9]+','',filename)
        if f_no_numbers == 'MEA__DIV__stim_addition.h':
            filenames.append(filename)

In [8]:
relevant_patterns = list(np.arange(100,122,2))

In [9]:
Parallel(n_jobs=10)(delayed(sort_data_by_pattern)(filename,relevant_patterns,True,-1) for filename in filenames)

[None, None, None, None]

## Noise Estimation

In [10]:
filenames = []
for (_,_,files) in os.walk(raw_data_path):
    for filename in files:
        f_no_numbers = re.sub(r'[0-9]+','',filename)
        if f_no_numbers == 'MEA__DIV__noise_estimation.h':
            filenames.append(filename)

In [11]:
relevant_patterns = [0,1]

In [12]:
Parallel(n_jobs=10)(delayed(sort_data_by_pattern)(filename,relevant_patterns,True,-1) for filename in filenames)

[None, None, None, None]